In [78]:
%matplotlib inline
import matplotlib.pyplot as plt
from astropy import units as u
from sedfitter import fit
from sedfitter.extinction import Extinction
import os

In [79]:
# path to models
model_dir = '/Users/yaolun/data/models_r06/'

# Read in extinction
dust_path = '/Users/yaolun/Google Drive/dust_model/Weingartner_Draine/kext_albedo_WD_MW_5.5A_30_D03.ext'
extinction = Extinction.from_file(dust_path, columns=[0,1], wav_unit=u.micron, chi_unit=u.cm**2/u.g)

dust_path = '/Users/yaolun/Google Drive/dust_model/kmh94.par'
extinction = Extinction.from_file(dust_path, columns=[0, 3],
                                  wav_unit=u.micron, chi_unit=u.cm**2 / u.g)


In [80]:
# set up the filters as the extraction of spectro-photometric fluxes
def filter_hyperion(wave):
    from sedfitter.filter import Filter
    import numpy as np
    import astropy.constants as const
    c = const.c.cgs.value
    
    if (wave < 50.) & (wave >= 5.):
        res = 60.
    elif wave < 5.:
        res = 10.
    else:
        res = 1000.
    
    wave_min = wave*(1-1./res)
    wave_max = wave*(1+1./res)
    
    if wave not in [70, 100, 160, 250, 350, 500]:
        f = Filter()
        f.name = str(wave)+'um'
        f.central_wavelength = wave*u.micron
        f.nu = (np.linspace(wave_min, wave_max, 100) * u.micron).to(u.Hz, equivalencies=u.spectral())
        f.response = np.ones_like(f.nu.value)
        f.normalize()
    # herschel filters
    else:
        import sys
        sys.path.append('/Users/yaolun/programs/misc/hyperion/')
        from phot_filter import phot_filter
        
        if wave == 70:
            fil_name = 'Herschel PACS 70um'
            name = 'P1'
        elif wave == 100:
            fil_name = 'Herschel PACS 100um'
            name = 'P2'
        elif wave == 160:
            fil_name = 'Herschel PACS 160um'
            name = 'P3'
        elif wave == 250:
            fil_name = 'Herschel SPIRE 250um'
            name = 'S1'
        elif wave == 350:
            fil_name = 'Herschel SPIRE 350um'
            name = 'S2'
        elif wave == 500:
            fil_name = 'Herschel SPIRE 500um'
            name = 'S3'
        
        print fil_name
        
        filter_func = phot_filter(fil_name, '/Users/yaolun/misc/hyperion/')
        
        f = Filter()
        f.name = name
        f.central_wavelength = wave * u.micron
        f.nu = (filter_func['wave'] * u.micron).to(u.Hz, equivalencies=u.spectral())
        f.response = filter_func['transmission']
    return f

In [ ]:
# monochromatic convolution on models
from sedfitter import convolve
mono_wav = [8.5, 9, 9.7, 10, 10.5, 11, 16, 20, 35]
mono_filters = []

for iwav in mono_wav:
    filter_dum = filter_hyperion(iwav)
    if not os.path.exists(model_dir+'/convolved/'+filter_dum.name+'.fits'):
        mono_filters.append(filter_dum)

In [ ]:
# Herschel filters convolution on models
herschel_wave = [70, 100, 160, 250, 350, 500]
herschel_filters = []

for iwav in herschel_wave:
    filter_dum = filter_hyperion(iwav)
    if not os.path.exists(model_dir+'/convolved/'+filter_dum.name+'.fits'):
        herschel_filters.append(filter_dum)

In [ ]:
convolve.convolve_model_dir(model_dir,mono_filters)

## Try to fit the SED without invoking any additional convolution

### Read in the test data

In [95]:
# Define filters and apertures
filters = ['I3','I4','8.5um','9um','9.7um',
           '10um','10.5um','11um','16um','20um',
           'M1','35um','P1','P2','P3',
           'HS1','HS2','HS3']
apertures = [7.2, 7.2, 7.2, 7.2, 7.2,
             7.2, 7.2, 7.2, 20.4, 20.4,
             20.4, 20.4, 31.8, 31.8, 31.8,
             35.4, 47.9, 46.0] * u.arcsec

fit('/Users/yaolun/bhr71/best_calibrated/bhr71_data_for_sedfitter_test.txt', filters, apertures, model_dir,
    'output.fitinfo', extinction_law=extinction, distance_range=[100, 300] * u.pc, av_range=[0, 20])

 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  YSO models
   Log[d] stepping     :  0.01
   Number of distances :  48

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /Users/yaolun/data/models_r06//convolved/I3.fits
   Reading /Users/yaolun/data/models_r06//convolved/I4.fits
   Reading /Users/yaolun/data/models_r06//convolved/8.5um.fits
   Reading /Users/yaolun/data/models_r06//convolved/9um.fits
   Reading /Users/yaolun/data/models_r06//convolved/9.7um.fits
   Reading /Users/yaolun/data/models_r06//convolved/10um.fits
   Reading /Users/yaolun/data/models_r06//convolved/10.5um.fits
   Reading /Users/yaolun/data/models_r06//convolved/11um.fits
   Reading /Users/yaolun/data/models_r06//convolved/16um.fits
   Reading /Users/yaolun/data/models_r06

In [88]:
from sedfitter import write_parameters, write_parameter_ranges

# Write out all models with a delta chi^2-chi_best^2 per datapoint < 3
write_parameters('output.fitinfo', 'parameters.txt',
                 select_format=('F', 3.))


In [90]:
from sedfitter import plot
plot('output.fitinfo', 'plots_seds')

Delete directory plots_seds? [y/[n]] y



In [ ]:
params